In [91]:
import sys
import os

# Ruta al directorio raíz del proyecto (ajústala si es necesario)
ruta_raiz = os.path.abspath(r"C:\Users\Germán Llorente\Desktop\germiprogramer\TFG-Agente-de-Scouting")

if ruta_raiz not in sys.path:
    sys.path.append(ruta_raiz)

In [1]:
import importlib
import funciones_analisis.estadisticas

importlib.reload(funciones_analisis.estadisticas)
importlib.reload(funciones_analisis.generales)

from funciones_analisis.estadisticas import *
from funciones_analisis.generales import *
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects
import seaborn as sns
import glob

In [2]:
ratings = pd.read_csv("datos/ratings/data_football_ratings.csv")

In [94]:
ratings

,competition,date,match,team,pos,pos_role,player,rater,is_human,original_rating,...,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,win,lost,is_home_team,minutesPlayed,game_duration
0,Euro 2016,10/06/2016,"France - Romania, 2 - 1",Romania,DF,DC,Dragos Grigore,Kicker,1,3.50,...,0.143055,0.603571,0.304348,0.000000,0.000000,0,1,0,90,90
1,Euro 2016,10/06/2016,"France - Romania, 2 - 1",Romania,DF,DC,Dragos Grigore,WhoScored,0,6.56,...,0.143055,0.603571,0.304348,0.000000,0.000000,0,1,0,90,90
2,Euro 2016,10/06/2016,"France - Romania, 2 - 1",Romania,DF,DC,Dragos Grigore,SofaScore,0,6.70,...,0.143055,0.603571,0.304348,0.000000,0.000000,0,1,0,90,90
3,Euro 2016,10/06/2016,"France - Romania, 2 - 1",France,Sub,Sub,Anthony Martial,WhoScored,0,6.19,...,0.051556,0.524845,0.041096,0.166667,0.000000,1,0,1,13,90
4,Euro 2016,10/06/2016,"France - Romania, 2 - 1",Romania,MF,DMC,Mihai Pintilii,Kicker,1,3.50,...,0.333284,0.710084,0.347826,0.675075,0.000000,0,1,0,90,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50647,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,MF,ML,Ivan Perisic,SkySports,1,8.00,...,0.172090,0.791209,0.318681,0.794142,0.045455,0,1,0,90,90
50648,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,MF,ML,Ivan Perisic,WhoScored,0,6.89,...,0.172090,0.791209,0.318681,0.794142,0.045455,0,1,0,90,90
50649,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,MF,ML,Ivan Perisic,SofaScore,0,6.40,...,0.172090,0.791209,0.318681,0.794142,0.045455,0,1,0,90,90
50650,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,Sub,Sub,Marko Pjaca,SkySports,1,6.00,...,0.000000,0.481605,0.032967,0.333333,0.000000,0,1,0,90,90


In [95]:
ratings.columns

Index(['competition', 'date', 'match', 'team', 'pos', 'pos_role', 'player',
       'rater', 'is_human', 'original_rating', 'goals', 'assists',
       'shots_ontarget', 'shots_offtarget', 'shotsblocked', 'chances2score',
       'drib_success', 'drib_unsuccess', 'keypasses', 'touches', 'passes_acc',
       'passes_inacc', 'crosses_acc', 'crosses_inacc', 'lballs_acc',
       'lballs_inacc', 'grduels_w', 'grduels_l', 'aerials_w', 'aerials_l',
       'poss_lost', 'fouls', 'wasfouled', 'clearances', 'stop_shots',
       'interceptions', 'tackles', 'dribbled_past', 'tballs_acc',
       'tballs_inacc', 'ycards', 'rcards', 'dangmistakes', 'countattack',
       'offsides', 'goals_ag_otb', 'goals_ag_itb', 'saves_itb', 'saves_otb',
       'saved_pen', 'missed_penalties', 'owngoals', 'degree_centrality',
       'betweenness_centrality', 'closeness_centrality', 'flow_centrality',
       'flow_success', 'betweenness2goals', 'win', 'lost', 'is_home_team',
       'minutesPlayed', 'game_duration'],
    

In [5]:
ratings = ratings[ratings['competition'] == 'World Cup 2018']

In [6]:
ratings

,competition,date,match,team,pos,pos_role,player,rater,is_human,original_rating,...,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,win,lost,is_home_team,minutesPlayed,game_duration
44533,World Cup 2018,14/06/2018,"Russia - Saudi Arabia, 5 - 0",Russia,MF,AMR,Aleksandr Samedov,WhoScored,0,7.09,...,0.129639,0.670635,0.239437,0.741989,0.000000,1,0,1,64,90
44534,World Cup 2018,14/06/2018,"Russia - Saudi Arabia, 5 - 0",Russia,MF,AMR,Aleksandr Samedov,SkySports,1,7.00,...,0.129639,0.670635,0.239437,0.741989,0.000000,1,0,1,64,90
44535,World Cup 2018,14/06/2018,"Russia - Saudi Arabia, 5 - 0",Russia,MF,AMR,Aleksandr Samedov,SofaScore,0,6.90,...,0.129639,0.670635,0.239437,0.741989,0.000000,1,0,1,64,90
44536,World Cup 2018,14/06/2018,"Russia - Saudi Arabia, 5 - 0",Russia,MF,AMR,Aleksandr Samedov,Kicker,1,4.00,...,0.129639,0.670635,0.239437,0.741989,0.000000,1,0,1,64,90
44537,World Cup 2018,14/06/2018,"Russia - Saudi Arabia, 5 - 0",Russia,FW,FW,Fedor Smolov,WhoScored,0,6.57,...,0.119464,0.635338,0.225352,1.156960,0.000000,1,0,1,70,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50647,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,MF,ML,Ivan Perisic,SkySports,1,8.00,...,0.172090,0.791209,0.318681,0.794142,0.045455,0,1,0,90,90
50648,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,MF,ML,Ivan Perisic,WhoScored,0,6.89,...,0.172090,0.791209,0.318681,0.794142,0.045455,0,1,0,90,90
50649,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,MF,ML,Ivan Perisic,SofaScore,0,6.40,...,0.172090,0.791209,0.318681,0.794142,0.045455,0,1,0,90,90
50650,World Cup 2018,15/07/2018,"France - Croatia, 4 - 2",Croatia,Sub,Sub,Marko Pjaca,SkySports,1,6.00,...,0.000000,0.481605,0.032967,0.333333,0.000000,0,1,0,90,90


In [97]:
len(ratings["player"].unique())

591

In [98]:
# Agrupar por player y agregar: media de original_rating, y listas únicas de team, pos y pos_role
df_summary = ratings.groupby('player').agg({
    'original_rating': 'mean',
    'team': lambda x: ', '.join(sorted(set(x))),
    'pos': lambda x: ', '.join(sorted(set(x))),
    'pos_role': lambda x: ', '.join(sorted(set(x))),
}).reset_index()

# Renombrar la columna de media
df_summary.rename(columns={'original_rating': 'avg_rating'}, inplace=True)

df_summary

,player,avg_rating,team,pos,pos_role
0,Aaron Mooy,6.443333,Australia,MF,MC
1,Abdallah El Said,5.935833,Egypt,MF,"AMC, AMR"
2,Abdiel Arroyo,5.967500,Panama,Sub,Sub
3,Abdullah Al-Mayoof,5.062500,Saudi Arabia,GK,GK
4,Abdullah Otayf,6.224167,Saudi Arabia,MF,DMC
...,...,...,...,...,...
586,Yuri Zhirkov,5.980000,Russia,"DF, MF","DL, DML"
587,Yussuf Poulsen,6.052500,Denmark,MF,"AMR, MR"
588,Yuto Nagatomo,5.824375,Japan,DF,DL
589,Yuya Osako,5.926000,Japan,"FW, Sub","FW, Sub"


In [99]:
df_summary["team"].unique()

array(['Australia', 'Egypt', 'Panama', 'Saudi Arabia', 'Nigeria',
       'Colombia', 'Morocco', 'Serbia', 'Belgium', 'Portugal', 'Tunisia',
       'Russia', 'Iceland', 'Peru', 'Sweden', 'Senegal', 'Iran', 'Brazil',
       'Denmark', 'Croatia', 'Mexico', 'Spain', 'Argentina', 'France',
       'Germany', 'Poland', 'England', 'Switzerland', 'Costa Rica',
       'Colombia, Uruguay', 'South Korea', 'Uruguay', 'Japan'],
      dtype=object)

In [100]:
df_summary.to_csv("datos/ratings/ratings_limpiado.csv", index=False)

In [ ]:
free_comps = sb.competitions()

mundial_2018 = sb.matches(competition_id=43, season_id=3)

In [ ]:
mundial_2018

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,7585,2018-07-03,20:00:00.000,International - FIFA World Cup,2018,Colombia,England,1,1,available,...,2021-06-13T16:17:31.694,4,Round of 16,Otkritie Bank Arena,Mark Geiger,José Néstor Pekerman,Gareth Southgate,1.0.2,None,None
1,7570,2018-06-28,20:00:00.000,International - FIFA World Cup,2018,England,Belgium,0,1,available,...,2021-06-13T16:17:31.694,3,Group Stage,Stadion Kaliningrad,Damir Skomina,Gareth Southgate,Roberto Martínez Montoliú,1.0.2,None,None
2,7586,2018-07-03,16:00:00.000,International - FIFA World Cup,2018,Sweden,Switzerland,1,0,available,...,2021-06-13T16:17:31.694,4,Round of 16,Saint-Petersburg Stadium,Damir Skomina,Jan Olof Andersson,Vladimir Petković,1.0.2,None,None
3,7557,2018-06-25,20:00:00.000,International - FIFA World Cup,2018,Iran,Portugal,1,1,available,...,2021-06-13T16:17:31.694,3,Group Stage,Mordovia Arena,Enrique Cáceres,Carlos Manuel Brito Leal Queiróz,Fernando Manuel Fernandes da Costa Santos,1.0.2,None,None
4,7542,2018-06-20,14:00:00.000,International - FIFA World Cup,2018,Portugal,Morocco,1,0,available,...,2021-06-13T16:17:31.694,2,Group Stage,Stadion Luzhniki,Mark Geiger,Fernando Manuel Fernandes da Costa Santos,Hervé Renard,1.0.2,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7540,2018-06-19,20:00:00.000,International - FIFA World Cup,2018,Russia,Egypt,3,1,available,...,2021-06-13T16:17:31.694,2,Group Stage,Saint-Petersburg Stadium,Enrique Cáceres,Stanislav Cherchesov,Héctor Raúl Cúper,1.0.2,None,None
60,8652,2018-07-07,20:00:00.000,International - FIFA World Cup,2018,Russia,Croatia,2,2,available,...,2021-06-13T16:17:31.694,5,Quarter-finals,\tOlimpiyskiy Stadion Fisht,Sandro Ricci,Stanislav Cherchesov,Zlatko Dalić,1.0.2,None,None
61,7563,2018-06-26,16:00:00.000,International - FIFA World Cup,2018,Denmark,France,0,0,available,...,2021-06-13T16:17:31.694,3,Group Stage,Stadion Luzhniki,Sandro Ricci,Åge Fridtjof Hareide,Didier Deschamps,1.0.2,None,None
62,7556,2018-06-24,17:00:00.000,International - FIFA World Cup,2018,Japan,Senegal,2,2,available,...,2021-06-13T16:17:31.694,2,Group Stage,\tEkaterinburg Arena,Gianluca Rocchi,Akira Nishino,Aliou Cissé,1.0.2,None,None


In [ ]:
data = mundial_2018

In [ ]:
# Lista para guardar DataFrames de eventos
eventos_lista = []

# Descargar eventos de cada partido
for match_id in data['match_id']:

        eventos = sb.events(match_id=match_id)
        eventos['match_id'] = match_id  # Añadir identificador de partido
        eventos_lista.append(eventos)


# Unir todos los DataFrames en uno solo
eventos_data = pd.concat(eventos_lista, ignore_index=True)

# Guardar como archivo CSV
#df_eventos.to_csv('eventos_seriea_2016.csv', index=False)


In [ ]:
def perfil_jugadores(df_eventos):
    # Eliminar eventos sin jugador
    df = df_eventos.dropna(subset=['player_id', 'player']).copy()

    # Lista para guardar los resultados
    perfiles = []

    # Agrupar por jugador
    for player_id, grupo in df.groupby('player_id'):
        nombre = grupo['player'].iloc[-1]
        ultimo_equipo = grupo['team'].dropna().iloc[-1] if 'team' in grupo else None
        ultimo_team_id = grupo['team_id'].dropna().iloc[-1] if 'team_id' in grupo else None
        posiciones = grupo['position'].dropna().unique().tolist()
        partidos_jugados = grupo['match_id'].nunique()  # contar partidos únicos

        perfiles.append({
            'player_id': int(player_id),
            'player': nombre,
            'ultimo_equipo': ultimo_equipo,
            'ultimo_team_id': ultimo_team_id,
            'posiciones': posiciones,
            'partidos_jugados': partidos_jugados
        })

    return pd.DataFrame(perfiles)


perfiles_jugadores = perfil_jugadores(eventos_data)

In [ ]:
perfiles_jugadores

,player_id,player,ultimo_equipo,ultimo_team_id,posiciones,partidos_jugados
0,2941,Ismaïla Sarr,Senegal,787,[Right Wing],3
1,2948,Nabil Fekir,France,771,"[Center Attacking Midfield, Right Wing Back, L...",6
2,2954,Youri Tielemans,Belgium,782,"[Left Center Midfield, Right Attacking Midfiel...",4
3,2974,Saîf-Eddine Khaoui,Tunisia,777,[Left Center Midfield],1
4,2995,Ángel Fabián Di María Hernández,Argentina,779,"[Right Wing, Left Wing, Left Midfield, Left At...",3
...,...,...,...,...,...,...
598,19597,Marcos Javier Acuña,Argentina,779,"[Left Midfield, Left Back]",1
599,19988,Jimmy Durmaz,Sweden,790,[Right Midfield],1
600,20004,Paul Pogba,France,771,"[Left Center Midfield, Right Defensive Midfiel...",6
601,20005,Toby Alderweireld,Belgium,782,"[Right Center Back, Right Back]",6


In [ ]:
print(perfiles_jugadores["ultimo_equipo"].unique())

['Senegal' 'France' 'Belgium' 'Tunisia' 'Argentina' 'Denmark' 'Poland'
 'Brazil' 'Mexico' 'Spain' 'South Korea' 'England' 'Germany' 'Portugal'
 'Japan' 'Australia' 'Colombia' 'Morocco' 'Nigeria' 'Serbia' 'Croatia'
 'Switzerland' 'Egypt' 'Sweden' 'Peru' 'Iceland' 'Uruguay' 'Russia'
 'Saudi Arabia' 'Iran' 'Costa Rica' 'Panama']


In [ ]:
df_summary.head(2)

,player,avg_rating,team,pos,pos_role
0,Aaron Mooy,6.443333,Australia,MF,MC
1,Abdallah El Said,5.935833,Egypt,MF,"AMC, AMR"


In [ ]:
perfiles_jugadores[perfiles_jugadores["ultimo_equipo"] == "Argentina"]

,player_id,player,ultimo_equipo,ultimo_team_id,posiciones,partidos_jugados
4,2995,Ángel Fabián Di María Hernández,Argentina,779,"[Right Wing, Left Wing, Left Midfield, Left At...",3
18,3090,Nicolás Hernán Otamendi,Argentina,779,"[Right Center Back, Left Center Back, Center B...",4
34,3237,Sergio Leonel Agüero del Castillo,Argentina,779,"[Center Forward, Left Center Forward, Left Back]",4
87,3602,Faustino Marcos Alberto Rojo,Argentina,779,"[Left Center Back, Left Back]",3
98,3659,Wilfredo Daniel Caballero,Argentina,779,[Goalkeeper],2
274,5496,Cristian David Pavón,Argentina,779,"[Left Wing, Right Wing, Right Midfield, Left M...",4
275,5497,Gonzalo Gerardo Higuaín,Argentina,779,"[Center Forward, Right Center Forward, Left Ce...",3
277,5499,Maximiliano Eduardo Meza,Argentina,779,"[Right Wing, Left Attacking Midfield, Left Mid...",4
281,5503,Lionel Andrés Messi Cuccittini,Argentina,779,"[Center Forward, Secondary Striker, Right Atta...",4
282,5504,Éver Maximiliano David Banega,Argentina,779,"[Left Center Midfield, Right Center Midfield, ...",3


In [ ]:

from rapidfuzz import process, fuzz

def hacer_matching_aproximado_por_pais(df_ratings, df_statsbomb, umbral=85, alias_manual=None):
    """
    Empareja jugadores del dataset de ratings con los de StatsBomb 
    usando similitud de nombres, priorizando coincidencias exactas definidas en alias_manual.
    """
    resultados = []

    for _, row in df_ratings.iterrows():
        nombre_jugador = row['player']
        pais_jugador = row['team']

        # 1. Buscar en alias manual
        if alias_manual and nombre_jugador in alias_manual:
            mejor_match = alias_manual[nombre_jugador]
            score = 100
        else:
            # 2. Matching aproximado por país
            candidatos = df_statsbomb[df_statsbomb['ultimo_equipo'] == pais_jugador]
            nombres_candidatos = candidatos['player'].unique().tolist()

            mejor_match, score, _ = process.extractOne(
                nombre_jugador,
                nombres_candidatos,
                scorer=fuzz.token_sort_ratio
            ) or (None, 0, None)

        resultados.append({
            'player_rating': nombre_jugador,
            'team': pais_jugador,
            'player_match_statsbomb': mejor_match,
            'score': score,
            'revisar': score < umbral and nombre_jugador not in alias_manual
        })

    return pd.DataFrame(resultados)


In [102]:
alias_manual = {
    "Angel Di Maria": "Ángel Fabián Di María Hernández"
}

df_matching = hacer_matching_aproximado_por_pais(df_summary, perfiles_jugadores, umbral=85, alias_manual=alias_manual)

df_ratings_matched = df_summary.merge(
    df_matching,
    left_on=['player', 'team'],
    right_on=['player_rating', 'team'],
    how='left'
)

df_final = df_ratings_matched.merge(
    perfiles_jugadores,
    left_on=['player_match_statsbomb', 'team'],
    right_on=['player', 'ultimo_equipo'],
    how='left',
    suffixes=('', '_statsbomb')
)


In [103]:
df_final

,player,avg_rating,team,pos,pos_role,player_rating,player_match_statsbomb,score,revisar,player_id,player_statsbomb,ultimo_equipo,ultimo_team_id,posiciones,partidos_jugados
0,Aaron Mooy,6.443333,Australia,MF,MC,Aaron Mooy,Aaron Mooy,100.000000,False,3281.0,Aaron Mooy,Australia,792.0,"[Left Center Midfield, Right Center Midfield]",3.0
1,Abdallah El Said,5.935833,Egypt,MF,"AMC, AMR",Abdallah El Said,Abdalla Mahmoud El Said Bekhit,69.565217,True,5251.0,Abdalla Mahmoud El Said Bekhit,Egypt,774.0,"[Center Attacking Midfield, Left Center Midfie...",3.0
2,Abdiel Arroyo,5.967500,Panama,Sub,Sub,Abdiel Arroyo,Abdiel Arroyo Molinar,76.470588,True,6140.0,Abdiel Arroyo Molinar,Panama,798.0,"[Right Wing, Left Center Midfield]",2.0
3,Abdullah Al-Mayoof,5.062500,Saudi Arabia,GK,GK,Abdullah Al-Mayoof,Abdullah Ibrahim Al Maiouf,68.181818,True,5189.0,Abdullah Ibrahim Al Maiouf,Saudi Arabia,799.0,[Goalkeeper],1.0
4,Abdullah Otayf,6.224167,Saudi Arabia,MF,DMC,Abdullah Otayf,Abdullah Ibrahim Otayf,77.777778,True,5173.0,Abdullah Ibrahim Otayf,Saudi Arabia,799.0,"[Left Center Midfield, Center Defensive Midfield]",3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,Yuri Zhirkov,5.980000,Russia,"DF, MF","DL, DML",Yuri Zhirkov,Yuri Zhirkov,100.000000,False,5193.0,Yuri Zhirkov,Russia,796.0,"[Left Midfield, Left Back]",3.0
587,Yussuf Poulsen,6.052500,Denmark,MF,"AMR, MR",Yussuf Poulsen,Yussuf Yurary Poulsen,80.000000,True,5536.0,Yussuf Yurary Poulsen,Denmark,776.0,[Right Wing],3.0
588,Yuto Nagatomo,5.824375,Japan,DF,DL,Yuto Nagatomo,Yuto Nagatomo,100.000000,False,5690.0,Yuto Nagatomo,Japan,778.0,[Left Back],4.0
589,Yuya Osako,5.926000,Japan,"FW, Sub","FW, Sub",Yuya Osako,Yuto Nagatomo,43.478261,True,5690.0,Yuto Nagatomo,Japan,778.0,[Left Back],4.0


In [104]:
df_final.columns

Index(['player', 'avg_rating', 'team', 'pos', 'pos_role', 'player_rating',
       'player_match_statsbomb', 'score', 'revisar', 'player_id',
       'player_statsbomb', 'ultimo_equipo', 'ultimo_team_id', 'posiciones',
       'partidos_jugados'],
      dtype='object')

In [105]:
# Eliminar columnas intermedias no necesarias
columnas_a_eliminar = ['player_rating', 'team', 'pos', 'pos_role', "player_statsbomb"]
df_final = df_final.drop(columns=columnas_a_eliminar, errors='ignore')

# Renombrar la columna final del jugador
df_final.rename(columns={'player_match_statsbomb': 'player_2'}, inplace=True)

In [106]:
df_final

,player,avg_rating,player_2,score,revisar,player_id,ultimo_equipo,ultimo_team_id,posiciones,partidos_jugados
0,Aaron Mooy,6.443333,Aaron Mooy,100.000000,False,3281.0,Australia,792.0,"[Left Center Midfield, Right Center Midfield]",3.0
1,Abdallah El Said,5.935833,Abdalla Mahmoud El Said Bekhit,69.565217,True,5251.0,Egypt,774.0,"[Center Attacking Midfield, Left Center Midfie...",3.0
2,Abdiel Arroyo,5.967500,Abdiel Arroyo Molinar,76.470588,True,6140.0,Panama,798.0,"[Right Wing, Left Center Midfield]",2.0
3,Abdullah Al-Mayoof,5.062500,Abdullah Ibrahim Al Maiouf,68.181818,True,5189.0,Saudi Arabia,799.0,[Goalkeeper],1.0
4,Abdullah Otayf,6.224167,Abdullah Ibrahim Otayf,77.777778,True,5173.0,Saudi Arabia,799.0,"[Left Center Midfield, Center Defensive Midfield]",3.0
...,...,...,...,...,...,...,...,...,...,...
586,Yuri Zhirkov,5.980000,Yuri Zhirkov,100.000000,False,5193.0,Russia,796.0,"[Left Midfield, Left Back]",3.0
587,Yussuf Poulsen,6.052500,Yussuf Yurary Poulsen,80.000000,True,5536.0,Denmark,776.0,[Right Wing],3.0
588,Yuto Nagatomo,5.824375,Yuto Nagatomo,100.000000,False,5690.0,Japan,778.0,[Left Back],4.0
589,Yuya Osako,5.926000,Yuto Nagatomo,43.478261,True,5690.0,Japan,778.0,[Left Back],4.0


In [108]:
# Lista de jugadores que quieres eliminar
jugadores_erroneos = [
    "Carlos Sanchez", "Koke", "Pepe", "Isco", "Nacho", "Kahraba", "Trezeguet",
    "Chicharito", "Zanka", "Marquinhos", "Paulinho", "Joao Mario", "Edison Flores",
    "Marcelo", "Edson Alvarez", "Yuya Osako", "Vedran Corluka", "Steven Nzonzi",
    "Kari Arnason", "Cho Hyun-Woo", "Badou Ndiaye", "Danny Welbeck"
]

# Eliminar esas filas del DataFrame
df_final = df_final[~df_final['player'].isin(jugadores_erroneos)]


In [109]:
# Eliminar columnas intermedias no necesarias
columnas_a_eliminar = ['player']
df_final = df_final.drop(columns=columnas_a_eliminar, errors='ignore')

# Renombrar la columna final del jugador
df_final.rename(columns={'player_2': 'player'}, inplace=True)

In [110]:
df_final

,avg_rating,player,score,revisar,player_id,ultimo_equipo,ultimo_team_id,posiciones,partidos_jugados
0,6.443333,Aaron Mooy,100.000000,False,3281.0,Australia,792.0,"[Left Center Midfield, Right Center Midfield]",3.0
1,5.935833,Abdalla Mahmoud El Said Bekhit,69.565217,True,5251.0,Egypt,774.0,"[Center Attacking Midfield, Left Center Midfie...",3.0
2,5.967500,Abdiel Arroyo Molinar,76.470588,True,6140.0,Panama,798.0,"[Right Wing, Left Center Midfield]",2.0
3,5.062500,Abdullah Ibrahim Al Maiouf,68.181818,True,5189.0,Saudi Arabia,799.0,[Goalkeeper],1.0
4,6.224167,Abdullah Ibrahim Otayf,77.777778,True,5173.0,Saudi Arabia,799.0,"[Left Center Midfield, Center Defensive Midfield]",3.0
...,...,...,...,...,...,...,...,...,...
584,6.355000,Young-Sun Yun,100.000000,False,6323.0,South Korea,791.0,[Right Center Back],1.0
585,5.932500,Yury Gazinskiy,92.857143,False,5181.0,Russia,796.0,"[Left Defensive Midfield, Right Defensive Midf...",4.0
586,5.980000,Yuri Zhirkov,100.000000,False,5193.0,Russia,796.0,"[Left Midfield, Left Back]",3.0
587,6.052500,Yussuf Yurary Poulsen,80.000000,True,5536.0,Denmark,776.0,[Right Wing],3.0


In [111]:
df_final.to_csv("datos/ratings/ratings_mundial_2018.csv")